In [1]:
using PowerModels

In [2]:
pwd()

"C:\\Users\\firda\\belajar\\ISE711"

In [4]:
filename =  "julia/pglib-opf-17.08/pglib_opf_case30_ieee.m"
network_data = PowerModels.parse_file(filename)


[warn | PowerModels]: the voltage setpoint on generator 4 does not match the value at bus 8
[warn | PowerModels]: the voltage setpoint on generator 1 does not match the value at bus 1
[warn | PowerModels]: the voltage setpoint on generator 5 does not match the value at bus 11
[warn | PowerModels]: the voltage setpoint on generator 2 does not match the value at bus 2
[warn | PowerModels]: the voltage setpoint on generator 6 does not match the value at bus 13
[warn | PowerModels]: the voltage setpoint on generator 3 does not match the value at bus 5
[info | PowerModels]: removing 3 cost terms from generator 4: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 1: [3211.2558999999997, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 5: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 2: [6179.8693, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 6: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 3: F

In [5]:
println(network_data["bus"]["1"])

Dict{String, Any}("zone" => 1, "bus_i" => 1, "bus_type" => 3, "vmax" => 1.06, "source_id" => Any["bus", 1], "area" => 1, "vmin" => 0.94, "index" => 1, "va" => 0.0, "vm" => 1.0, "base_kv" => 132.0)
